# My Playground for Neural Network

In [325]:
from utils import *
from stratified import *
from run import *

In [326]:
layerdataparameter = [4,5,4] # [First layer, Second layer, Third layer]

In [327]:
def importhousedata():
    house = importfile('hw3_house_votes_84.csv', ',')
    housecategory = {}
    for i in house[0]:
        housecategory[i] = 'categorical'
    housecategory["class"] = 'class'
    housedata = np.array(house[1:]).astype(float)
    return housedata, housecategory

In [328]:
from sklearn.preprocessing import OneHotEncoder
house_data, house_category = importhousedata()
example_category = {'ece': 'categorical','cs': 'categorical','phy': 'categorical','math': 'categorical','musi': 'categorical','grade': 'class'}
example_data = np.array([[1,0,1,1,0,0],[1,1,0,1,1,1],[0,1,0,0,0,2]])

In [329]:
def onehotencoder(data,category):
    dataT=data.T.copy()
    enc = OneHotEncoder(sparse=False)
    i = 0
    appendeddict = {}
    for cat in category:
        if category[cat] == 'categorical':
            hotneeded = dataT[i]
            hotted = enc.fit_transform(hotneeded.reshape(-1,1))
            for j in enc.categories_[0]:
                newname = cat+'_'+str(j)
                appendeddict[newname] = 'numerical'
            dataT = np.append(dataT,hotted.T,axis=0)
        if category[cat] == 'class':
            hotneeded = dataT[i]
            hotted = enc.fit_transform(hotneeded.reshape(-1,1))
            for class_ in enc.categories_[0]:
                newname = cat+'_'+str(class_)
                appendeddict[newname] = 'class_numerical'
            dataT = np.append(dataT,hotted.T,axis=0)
        i += 1
    
    category.update(appendeddict)
    i = 0
    categorycopy = category.copy()
    droplist = []
    for cat in category:
        if category[cat] == 'categorical' or category[cat] == 'class':
            droplist.append(i)
            categorycopy.pop(cat)
        i += 1
    dataT = np.delete(dataT,droplist,axis=0)
    return dataT.T, categorycopy
            

In [330]:
ohe_house_data, ohe_house_category = onehotencoder(house_data,house_category)


In [331]:
print(len(ohe_house_data.T))
print(len(ohe_house_category))

50
50


In [332]:
enc = OneHotEncoder(sparse=False)
empty = []
example2 = np.array([1,0,1,2,0,1])
exed= example2.reshape(1,-1)
hotted = enc.fit_transform(example2.reshape(-1,1))
empty.append(hotted.T)
print(hotted.T)
print(empty)
for i in enc.categories_[0]:
    print(i)




[[0. 1. 0. 0. 1. 0.]
 [1. 0. 1. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0.]]
[array([[0., 1., 0., 0., 1., 0.],
       [1., 0., 1., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0.]])]
0
1
2
